In [8]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
!pip install nltk
!pip install stop_words
!pip install gensim
!pip install pyLDAvis

In [1]:
#!/usr/bin/envpython3
# coding: utf-8

### Latent Dirichlet Allocation (LDA)

In [2]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem import RSLPStemmer
import unicodedata
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

- Tokenização: separar palavras ou sentenças em unidades. Marca cada palavra como um token no texto.
- Stemmer: reduzir uma palavra ao seu radical

In [3]:
tokenizer = RegexpTokenizer(r'\w+')

# create Portuguese stop words list
pt_stop = get_stop_words('pt')

# Create p_stemmer of class PorterStemmer
p_stemmer = RSLPStemmer()

#### - Docs

In [13]:
# create sample documents
doc_a = "O brócolis é bom para comer. Meu irmão gosta de comer bons brócolis, mas não minha mãe."
doc_b = "Minha mãe passa muito tempo dirigindo para levar meu irmão para praticar futebol."
doc_c = "Alguns especialistas em saúde sugerem que a direção pode aumentar a tensão e a pressão sanguínea."
doc_d = "Costumo sentir pressão para me sair bem na escola, mas minha mãe parece nunca cobrar meu irmão para fazer melhor."
doc_e = "Profissionais da saúde dizem que brócolis faz bem a sua saúde."


In [14]:
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

In [21]:
import csv
import re
doc_set = []
with open('data.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        doc_set.append(row[10])
size = int(len(doc_set)/2)
print(size)
doc_set = doc_set[size:]

2603


#### - Pré Processamento

In [49]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:

    # clean and tokenize document string
    raw = i.lower()
    #data = text
    raw = unicodedata.normalize('NFKD', raw).encode('ASCII', 'ignore')
    raw = (str(raw).replace('b\'','').replace('\'',''))
    raw = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', raw, flags=re.MULTILINE)
    raw = re.sub(r'#(\w)+', '', raw, flags=re.MULTILINE)

    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in pt_stop and len(i) > 3 and 'source' not in i and 'youtube' not in i and not 'ig' in i and 'bolsonaro' not in i]

    # stem tokens
    #stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    #print(stemmed_tokens)

    # add tokens to list
    texts.append(stopped_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

#### LDA

In [50]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word = dictionary, passes=20,random_state=1)
#print(ldamodel)
print(ldamodel.print_topics(num_topics=4, num_words=4))

[(0, '0.027*"brasil" + 0.019*"twitter" + 0.018*"todos" + 0.015*"tudo"'), (1, '0.115*"elesim" + 0.026*"twitter" + 0.012*"turno" + 0.011*"primeiro"'), (2, '0.029*"twitter" + 0.009*"amanha" + 0.009*"ciro" + 0.008*"voce"'), (3, '0.021*"video" + 0.014*"gostei" + 0.011*"vote" + 0.009*"politica"')]


In [51]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)